# 004 Sweeps

Develop code for sweeping a profile along an alignment.

In [2]:
import math
import os

import pandas as pd

import ifcopenshell
import ifcopenshell.geom
from ifcopenshell.ifcgeom import Alignment


file_path = os.path.join(
    "..",
    "tests",
    "data",
)

test_file = "4REN0_Autodesk.ifc"
in_file = os.path.join(file_path, test_file)

model = ifcopenshell.open(in_file)

ent = model.by_type("IfcAlignment")[0]

align = Alignment().from_entity(ent)
pts = align.create_shape(use_representation=False, point_interval=25)
cols = ["Distance", "X", "Y", "Direction", "Z", "Cant", "Cant_Rotation", "ReferenceZ"]
df = pd.DataFrame(pts, columns=cols)

# confirm that P.T. of 1st horizontal curve is present in the distances array
df.head(21)



,Distance,X,Y,Direction,Z,Cant,Cant_Rotation,ReferenceZ
0,0.00000,0.269990,1291.933570,-0.742491,753.746629,NaN,NaN,NaN
1,25.00000,18.449292,1274.773392,-0.770645,753.103918,NaN,NaN,NaN
2,50.00000,36.138340,1257.108277,-0.798798,752.461206,NaN,NaN,NaN
3,75.00000,53.323116,1238.952226,-0.826951,751.818494,NaN,NaN,NaN
4,100.00000,69.990000,1220.319628,-0.855104,751.175782,NaN,NaN,NaN
5,125.00000,86.125782,1201.225250,-0.883257,750.533070,NaN,NaN,NaN
6,150.00000,101.717673,1181.684226,-0.911410,749.890359,NaN,NaN,NaN
7,175.00000,116.753317,1161.712042,-0.939564,749.247647,NaN,NaN,NaN
8,200.00000,131.220798,1141.324528,-0.967717,748.604935,NaN,NaN,NaN
9,225.00000,145.108648,1120.537841,-0.995870,747.962223,NaN,NaN,NaN


The length of the first curve is 484.31607, and it can be seen as element 20
in the dataframe.

## Building the sweep

We will use a single profile to generate a sweep of a pavement section.

This project utilized a single 16 ft travel lane, with the alignment on
the right edge of pavement.

IIRC the pavement thickness was 11 inches, or 0.91667 ft.


In [4]:
from OCC.Core.gp import gp_OZ
from OCC.Core.gp import gp_Pnt
from OCC.Core.gp import gp_Trsf
from OCC.Core.gp import gp_Vec
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeWire
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform
from OCC.Core.BRepOffsetAPI import BRepOffsetAPI_ThruSections

p1 = gp_Pnt(0, 0, 0)
p2 = gp_Pnt(-16, 0, 0)
p3 = gp_Pnt(-16, 0, -0.91667)
p4 = gp_Pnt(0, 0, -0.91667)

e1 = BRepBuilderAPI_MakeEdge(p1, p2)
e2 = BRepBuilderAPI_MakeEdge(p2, p3)
e3 = BRepBuilderAPI_MakeEdge(p3, p4)
e4 = BRepBuilderAPI_MakeEdge(p4, p1)

ramp_lane = BRepBuilderAPI_MakeWire()
ramp_lane.Add(e1.Edge())
ramp_lane.Add(e2.Edge())
ramp_lane.Add(e3.Edge())
ramp_lane.Add(e4.Edge())




Now that we have the profile as a wire,
we need to place the profile at each point along the alignment.

This is done by defining a transform of translation and rotation.

The translation is defined by the XYZ coordinates of each point.

The rotation is defined by the `direction` of each point,
which is a rotation about the global Z axis.

In [5]:

def transform_profile(translation_vec, rotation_angle, profile):
    move = gp_Trsf()
    rotate = gp_Trsf()
    move.SetTranslation(translation_vec)
    rotate.SetRotation(gp_OZ(), -(math.pi / 2) + rotation_angle)
    r = BRepBuilderAPI_Transform(profile.Wire(), rotate)
    m = BRepBuilderAPI_Transform(r.Shape(), move)

    return m.Shape()

def make_sweep(points, profile):
    """
    Sweep the profile along the calculated alignment points
    """
    sweep = BRepOffsetAPI_ThruSections(True, True)

    for p in points:
        X = p[1]
        Y = p[2]
        Z = p[4]
        trans = gp_Vec(X, Y, Z)
        rot = p[3]
        sweep.AddWire(transform_profile(trans, rot, profile))

    return sweep


corridor = make_sweep(pts, ramp_lane)
    

In [6]:
# GLTF export
from OCC.Core.Message import Message_ProgressRange
from OCC.Core.TDocStd import TDocStd_Document
from OCC.Core.TColStd import TColStd_IndexedDataMapOfStringString
from OCC.Core.TCollection import TCollection_AsciiString
from OCC.Core.TCollection import TCollection_ExtendedString
from OCC.Core.XCAFDoc import (
    XCAFDoc_DocumentTool_ShapeTool,
    XCAFDoc_DocumentTool_LayerTool,
)
from OCC.Core.BRepMesh import BRepMesh_IncrementalMesh
from OCC.Core.BRepTools import breptools_Clean
from OCC.Core.RWGltf import RWGltf_CafWriter, RWGltf_WriterTrsfFormat
from OCC.Core.RWMesh import RWMesh_CoordinateSystemConverter, RWMesh_CoordinateSystem_glTF, RWMesh_CoordinateSystem_Zup

shp = corridor.Shape()
doc = TDocStd_Document(TCollection_ExtendedString("pythonocc-doc"))
shape_tool = XCAFDoc_DocumentTool_ShapeTool(doc.Main())
layer_tool = XCAFDoc_DocumentTool_LayerTool(doc.Main())

# mesh shape
breptools_Clean(shp)
# Triangulate
msh_algo = BRepMesh_IncrementalMesh(shp, True)
msh_algo.Perform()

sub_shape_label = shape_tool.AddShape(shp)

# GLTF options
a_format = RWGltf_WriterTrsfFormat.RWGltf_WriterTrsfFormat_Compact
coord_sys_converter = RWMesh_CoordinateSystemConverter()
coord_sys_converter.SetInputCoordinateSystem(RWMesh_CoordinateSystem_Zup)
coord_sys_converter.SetOutputCoordinateSystem(RWMesh_CoordinateSystem_glTF)
force_uv_export = True

# metadata
a_file_info = TColStd_IndexedDataMapOfStringString()
a_file_info.Add(
    TCollection_AsciiString("Authors"), TCollection_AsciiString("ifcopenshell")
)

#
# Binary export
#
out_path = os.path.join("..", "out", "glTF")
out_file = os.path.join(out_path, "004-sweeps.glb")
binary = True
binary_rwgltf_writer = RWGltf_CafWriter(TCollection_AsciiString(out_file), binary)
binary_rwgltf_writer.SetTransformationFormat(a_format)
binary_rwgltf_writer.SetForcedUVExport(force_uv_export)
binary_rwgltf_writer.SetCoordinateSystemConverter(coord_sys_converter)
pr = Message_ProgressRange()  # this is required
binary_rwgltf_writer.Perform(doc, a_file_info, pr)

True